In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import ranksums,pearsonr
from sklearn.linear_model import LogisticRegression
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
GENERIC = "gap"
PREDICTOR = "monthly_pct_ret"

In [3]:
df = pd.read_csv("ratio_order_affinity_"+GENERIC+".csv")

df.dropna(inplace=True)

split = np.floor(0.75*df.shape[0]).astype(int)

df.sort_values(by='sunday_beginning',inplace=True)

#df['is_DEC'] = df['is_DEC'].astype(int)
#df['is_NOV'] = df['is_NOV'].astype(int)

#x = df[['six','five','four','three','two','one','is_NOV','is_DEC']]
x = df
x['ratio_L3'] = (x['ratio_one']+x['ratio_two']+x['ratio_three'])/3.0
x['ratio_L5'] = (x['ratio_one']+x['ratio_two']+x['ratio_three']+x['ratio_four']+x['ratio_five'])/5.0
print(x.head())
y = df[[PREDICTOR]]


#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)# random_state=321)



x_train = x.iloc[:split,:]
x_test = x.iloc[split+1:,:]
y_train = y.iloc[:split,:]
y_test = y.iloc[split+1:,:]


x_train1 = x_train[['ratio_six','ratio_five','ratio_four','ratio_three','ratio_two','ratio_one','ratio_L3','ratio_L5']]#,'is_NOV','is_DEC']]
x_test1 = x_test[['ratio_six','ratio_five','ratio_four','ratio_three','ratio_two','ratio_one','ratio_L3','ratio_L5']]#,'is_NOV','is_DEC']]

y_train_bin = (y_train>0).astype(int)
y_test_bin = (y_test>0).astype(int)

print x_train1.shape
print y_train_bin.shape

IOError: File ratio_order_affinity_gap.csv does not exist

In [ ]:
#df = pd.read_csv('nordstrom_reg_ready.csv')
#
#df.dropna(inplace=True)
#
#df['is_DEC'] = df['is_DEC'].astype(int)
#df['is_NOV'] = df['is_NOV'].astype(int)
#
#x = df[['six','five','four','three','two','one','is_NOV','is_DEC']]
#y = df[['monthly_ret']]
#
#x_train = x.iloc[:80,:]
#x_test = x.iloc[83:,:]
#y_train = y.iloc[:80,:]
#y_test = y.iloc[83:,:]
#
#static_train = x_train[['is_NOV','is_DEC']]
#static_test = x_test[['is_NOV','is_DEC']]
#
#x_train = x_train[['six','five','four','three','two','one']]
#x_test = x_test[['six','five','four','three','two','one']]
#
#x_train = np.array(x_train).reshape((-1,6,1))
#x_test = np.array(x_test).reshape((-1,6,1))


In [ ]:
model = LogisticRegression(class_weight='balanced')
model.fit(x_train1,y_train_bin)

In [ ]:
preds = model.predict_proba(x_test1)
preds = pd.DataFrame(preds)
y_test.reset_index(drop=True,inplace=True)
preds.head()

In [ ]:
y_test['preds'] = preds[1]
y_test['binary_ret'] = (y_test[PREDICTOR]>0).astype(int)

In [ ]:
y_test.sort_values(by='preds')

In [ ]:
pearsonr(y_test[PREDICTOR],y_test['preds'])

In [ ]:
x_test.reset_index(inplace=True)
x_test['preds'] = y_test['preds']

x_test.to_csv('testset_monthly.csv',index=False)

In [ ]:
preds = model.predict_proba(x_train1)
preds = pd.DataFrame(preds)
y_train.reset_index(drop=True,inplace=True)
y_train['preds'] = preds[1]
y_train['binary_ret'] = (y_train[PREDICTOR]>0).astype(int)

x_train.reset_index(inplace=True)
x_train['preds'] = y_train['preds']

x_train.to_csv('trainset_monthly.csv',index=False)

In [ ]:
print np.mean(y_test[PREDICTOR])
print "and the pct positive:"
print np.mean(y_test['binary_ret'])

In [ ]:
print np.count_nonzero(y_test[y_test['preds']<0.5][PREDICTOR])
print np.mean(y_test[y_test['preds']<0.5][PREDICTOR])

In [ ]:
a = np.linspace(0.50,0.7,5)

for pred in a:
    print "predicted return of "+str(pred)
    print 'count is: '+str(np.count_nonzero(y_test[y_test['preds']>pred][PREDICTOR]))
    print np.mean(y_test[y_test['preds']>pred][PREDICTOR])

In [ ]:
third = np.floor(y_test.shape[0]/3).astype(int)
third

In [ ]:
y_test1 = y_test.sort_values(by='preds')
y_test2 = y_test1.reset_index(drop=True)
print np.mean(y_test2.iloc[0:third][PREDICTOR])
print np.mean(y_test2.iloc[third+1:2*third][PREDICTOR])
print np.mean(y_test2.iloc[2*third+1:][PREDICTOR])
print "and now for medians:"
print np.median(y_test2.iloc[0:third][PREDICTOR])
print np.median(y_test2.iloc[third+1:2*third][PREDICTOR])
print np.median(y_test2.iloc[2*third+1:][PREDICTOR])

In [ ]:
ranksums(y_test[PREDICTOR],y_test['preds'])

In [ ]:
filename = GENERIC+'_logreg_temporal.sav'
pickle.dump(model, open(filename, 'wb'))